In [1]:
# Importando as bibliotecas
import bs4 
from bs4 import BeautifulSoup
import urllib.request as urllib_request
from urllib.request import Request, urlopen, urlretrieve
from urllib.request import Request, urlopen
from urllib.error import URLError, HTTPError
import pandas as pd

print("BeautifulSoup ->", bs4.__version__)
print("urllib ->", urllib_request.__version__)
print("pandas ->", pd.__version__)

BeautifulSoup -> 4.10.0
urllib -> 3.9
pandas -> 1.3.4


In [44]:
#Declarando variável cards
cards = []
garagem = []
quartos = []
rua = []
bairro = []

##Obtendo HTML e o total de paginas
pages = int(1)


##interando por todas as paginas do site
for i in range(pages):
    ##Obtendo o HTML
   
    response = 'https://www.zapimoveis.com.br/venda/casas/?pagina=' + str(i) + '&transacao=Venda&tipoUnidade=Residencial,Casa&tipo=Im%C3%B3vel%20usado'
    headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.71 Safari/537.36'}

    requisicao = Request(response,headers = headers)

    try:
        response = urlopen(requisicao)
        html = response.read().decode('utf-8')
        soup = BeautifulSoup(html, 'html.parser')
    except HTTPError as e:
        print(e.status(), e.reason())
    except URLError as e:
        print(e.reason()) 

    
    #Obtendo as Tags de interesse
    anuncios = soup.find('div',{'class':'listings__container'}).findAll('div',class_='card-listing simple-card')

    #Coletando as informações dos CARDS
    for anuncio in anuncios:
        card = {}

        # Valor
        valor = anuncio.find('p', {'class': 'simple-card__price js-price color-darker heading-regular heading-regular__bolder align-left'})
        if valor == None:
            card['valor'] = int('0')
        else :
            card['valor'] = int(valor.get_text().replace('R$','').replace('\n','').replace('.',''))
            
        
        #descricao
        descricao = anuncio.find('span',{'class':'simple-card__text text-regular'})
        if descricao == None:
            card['descricao'] = str('Vazio')
        else :
            card['descricao'] = str(descricao.get_text().replace('-',' ').replace('\n',''))


        #endereco
        endereco = anuncio.find('h2',{'class':'simple-card__address color-dark text-regular'})
        if endereco == None:
            card['local'], card['bairro/cidade'] = str('Não disponivel', 'Não disponivel')
        else :
            card['local'], card['bairro/cidade'], *outro = endereco.get_text().replace('\n','').replace('  ','').split(',')

            
        #quartos
        quartos = anuncio.find('span',{'itemprop':'numberOfRooms'})
        if quartos == None:
            card['quartos'] = int(0)
        else :
            card['quartos'] = int(quartos.get_text().replace(' ','').replace('\n',''))
        

        
        #vagas
        garagem = anuncio.find('li',{'class':'feature__item text-small js-parking-spaces'})
        if garagem == None:
            card['garagem'] = int(0)
        else :
            card['garagem'] = int(garagem.get_text().replace(' ','').replace('\n',''))

        
            
        #banheiros
        banheiro = anuncio.find('span',{'itemprop':'numberOfBathroomsTotal'})
        if banheiro == None:
            card['banheiro'] = int(0)
        else :
            card['banheiro'] = int(banheiro.get_text().replace(' ','').replace('\n',''))

          
        #metragem
        metro = anuncio.find('span',{'itemprop':'floorSize'})
        if metro == None:
            card['area(m2)'] = int(0)
        else :
            card['area(m2)'] = int(metro.get_text().replace(' ','').replace('\n','').replace('m²',''))


            
        #IPTU
        iptu = anuncio.find('li',{'class':'card-price__item iptu text-regular'})
        if iptu == None:
            card['iptu'] = int('0')
        else :
            card['iptu'] = int(iptu.get_text().replace('R$','').replace('\n','').replace('.','').replace('IPTU','').replace(' ',''))

                          
            
        #condominio
        condominio = anuncio.find('li',{'class':'card-price__item condominium text-regular'})
        if condominio == None:
            card['condominio'] = int('0')
        else :
            card['condominio'] = int(condominio.get_text().replace('condomínioR$','').replace(' ','').replace('.',''))
        
        
        #tipo
        tipo = anuncio.find('small')      
        if tipo == None:
            card['novidade'] = str('Nenhuma')
        else :
            card['novidade'] = str(tipo.get_text())


            
        cards.append(card)


        
dataset = pd.DataFrame(cards)
dataset.to_csv('./dados/dataset-casas-geral-teste.csv', sep=';' , index=False, encoding='utf-8-sig')
dataset

,valor,descricao,local,bairro/cidade,quartos,garagem,banheiro,area(m2),iptu,condominio,novidade
0,3900000,Pensando na funcionalidade para uma vida práti...,Vila Nova Conceição,São Paulo,3,2,2,215,400,0,Super destaque
1,259000,"Lindo Villagio com 03 sobrados, 2 dorms, cozin...",Jardim Esperança,Mogi das Cruzes,2,2,2,63,0,0,Super destaque
2,385000,Aconchegante sobrado pré fabricado com (03) tr...,Armação do Pântano do Sul,Florianópolis,3,1,2,98,0,0,Super destaque
3,300000,Conquiste seu imóvel próprio! Entrada facilit...,Rua Vereador Nesclar Faria Guimarães,Mogi Moderno,2,2,2,65,0,0,Destaque
4,95000,"A casa no bairro Prolar com 3 quartos, banheir...",Rua Alberto Paulo Thomaz,Prolar,3,1,1,300,0,0,Nenhuma
5,220000,Já adquiriu seu imóvel ! Ainda nãooooo!!!! Co...,Avenida Getúlio Vargas,Centro,2,2,1,180000,0,0,Nenhuma
6,250000,"Casa à venda no Bairro Dom Pedro I, excelente ...",Conjunto Residencial Dom Pedro I,São José dos Campos,1,3,2,85,296,0,Nenhuma
7,450000,O sobrado está localizado no bairro Centro com...,Centro,Guararema,3,2,3,150,0,0,Nenhuma
8,450000,Sobrado à venda Restinga SP . 200m² Terre...,,Restinga,3,3,3,350,0,0,Nenhuma
9,350000,"Casa com salão comercial com 3 dormitórios, se...",Centro,Serra Azul,3,1,3,120,0,0,Nenhuma


In [43]:
#Declarando variável cards
cards = []
garagem = []
quartos = []
rua = []
bairro = []

##Obtendo HTML e o total de paginas
pages = int(1)


##interando por todas as paginas do site
for i in range(pages):
    ##Obtendo o HTML
   
    response = 'https://www.zapimoveis.com.br/venda/apartamentos/?pagina='+str(i)
    headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.71 Safari/537.36'}

    requisicao = Request(response,headers = headers)

    try:
        response = urlopen(requisicao)
        html = response.read().decode('utf-8')
        soup = BeautifulSoup(html, 'html.parser')
    except HTTPError as e:
        print(e.status(), e.reason())
    except URLError as e:
        print(e.reason()) 

    
    #Obtendo as Tags de interesse
    anuncios = soup.find('div',{'class':'listings__container'}).findAll('div',class_='card-listing simple-card')

    #Coletando as informações dos CARDS
    for anuncio in anuncios:
        card = {}

        # Valor
        valor = anuncio.find('p', {'class': 'simple-card__price js-price color-darker heading-regular heading-regular__bolder align-left'})
        if valor == None:
            card['valor'] = int('0')
        else :
            card['valor'] = int(valor.get_text().replace('R$','').replace('\n','').replace('.',''))
            
        
        #descricao
        descricao = anuncio.find('span',{'class':'simple-card__text text-regular'})
        if descricao == None:
            card['descricao'] = str('Vazio')
        else :
            card['descricao'] = str(descricao.get_text().replace('-',' ').replace('\n',''))


        #endereco
        endereco = anuncio.find('h2',{'class':'simple-card__address color-dark text-regular'})
        if endereco == None:
            card['local'], card['bairro/cidade'] = 'Não disponivel', 'Não disponivel'
        else :
            card['local'], card['bairro/cidade'], *outro = endereco.get_text().replace('\n','').replace('  ','').split(',')

            
        #quartos
        quartos = anuncio.find('span',{'itemprop':'numberOfRooms'})
        if quartos == None:
            card['quartos'] = int(0)
        else :
            card['quartos'] = int(quartos.get_text().replace(' ','').replace('\n',''))
        

        
        #vagas
        garagem = anuncio.find('li',{'class':'feature__item text-small js-parking-spaces'})
        if garagem == None:
            card['garagem'] = int(0)
        else :
            card['garagem'] = int(garagem.get_text().replace(' ','').replace('\n',''))


        #banheiros
        banheiro = anuncio.find('span',{'itemprop':'numberOfBathroomsTotal'})
        if banheiro == None:
            card['banheiro'] = int(0)
        else :
            card['banheiro'] = int(banheiro.get_text().replace(' ','').replace('\n',''))

            
        #metragem
        metro = anuncio.find('span',{'itemprop':'floorSize'})
        if metro == None:
            card['area(m2)'] = int(0)
        else :
            card['area(m2)'] = int(metro.get_text().replace(' ','').replace('\n','').replace('m²',''))


        #IPTU
        iptu = anuncio.find('li',{'class':'card-price__item iptu text-regular'})
        if iptu == None:
            card['iptu'] = int('0')
        else :
            card['iptu'] = int(iptu.get_text().replace('R$','').replace('\n','').replace('.','').replace('IPTU','').replace(' ',''))


        #condominio
        condominio = anuncio.find('li',{'class':'card-price__item condominium text-regular'})
        if condominio == None:
            card['condominio'] = int('0')
        else :
            card['condominio'] = int(condominio.get_text().replace('condomínioR$','').replace(' ','').replace('.',''))
        
        #tipo
        tipo = anuncio.find('small')      
        if tipo == None:
            card['novidade'] = str('Nenhuma')
        else :
            card['novidade'] = str(tipo.get_text())
            
        cards.append(card)

        
dataset = pd.DataFrame(cards)
dataset.to_csv('./dados/dataset-ap-geral-teste.csv', sep=';', index=False, encoding='utf-8-sig')
dataset

,valor,descricao,local,bairro/cidade,quartos,garagem,banheiro,area(m2),iptu,condominio,novidade
0,350000,Semimobiliado Área útil de 102 metros quad...,Rua Emílio de Menezes,Cidade Ocian,3,1,1,102,160,737,Super destaque
1,160000,Apartamento semi mobiliado de baixo custo e ót...,Bom Viver,Biguaçu,2,1,1,48,0,181,Super destaque
2,919000,"Apartamento de 94m², com varanda gourmet com c...",Rua Faustolo,Vila Romana,3,2,3,94,5,11,Super destaque
3,268600,"São apartamentos de 2 Quartos com Suíte, Cozin...",Módulo 1,Condominio Residencial Santa Maria,2,1,2,53,250,250,Destaque
4,189000,"Um projeto moderno, envolvente e extremamente ...",Rua Cataguá,Jardim Santa Tereza,2,1,1,46,25,200,Nenhuma
5,268587,São apartamentos de 2 QUARTOS com sala com var...,Quadra 3,Setor Central,2,1,2,53,300,300,Nenhuma
6,320000,"Excelente localização, em rua sem saída, próxi...",Rua Doutor Artur Tibau,Ingá,2,0,2,70,100,600,Nenhuma
7,340000,O apartamento no bairro Loteamento Villa Branc...,Avenida das Letras,Villa Branca,2,2,1,68,0,0,Nenhuma
8,310000,CONHEÇA AGORA MESMO AS VANTAGENS EXCLUSIVAS QU...,CSG 3,Taguatinga Sul,2,1,1,50,300,300,Nenhuma
9,268600,"São apartamentos de 2 Quartos com Suíte, Cozin...",Módulo 1,Condominio Residencial Santa Maria,2,1,2,53,250,250,Nenhuma


In [41]:
#Declarando variável cards
cards = []
garagem = []
quartos = []
rua = []
bairro = []

##Obtendo HTML e o total de paginas
pages = int(1)


##interando por todas as paginas do site
for i in range(pages):
    
    ##Obtendo o HTML
    response = 'https://www.zapimoveis.com.br/venda/casas/rj+rio-de-janeiro/?pagina='+str(i)
    headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.71 Safari/537.36'}

    requisicao = Request(response,headers = headers)

    try:
        response = urlopen(requisicao)
        html = response.read().decode('utf-8')
        soup = BeautifulSoup(html, 'html.parser')
    except HTTPError as e:
        print(e.status(), e.reason())
    except URLError as e:
        print(e.reason()) 

    
    #Obtendo as Tags de interesse
    anuncios = soup.find('div',{'class':'listings__container'}).findAll('div',class_='simple-card__box')

    #Coletando as informações dos CARDS
    for anuncio in anuncios:
        card = {}

        # Valor
        valor = anuncio.find('p', {'class': 'simple-card__price js-price color-darker heading-regular heading-regular__bolder align-left'})
        if valor == None:
            card['valor'] = int('0')
        else :
            card['valor'] = int(valor.get_text().replace('R$','').replace('\n','').replace('.',''))
            
        
        #descricao
        descricao = anuncio.find('span',{'class':'simple-card__text text-regular'})
        if descricao == None:
            card['descricao'] = str('Vazio')
        else :
            card['descricao'] = str(descricao.get_text().replace('-',' ').replace('\n',''))


        #endereco
        endereco = anuncio.find('h2',{'class':'simple-card__address color-dark text-regular'})
        if endereco == None:
            card['local'], card['bairro/cidade'] = 'Não disponivel', 'Não disponivel'
        else :
            card['local'], card['bairro/cidade'], *outro = endereco.get_text().replace('\n','').replace('  ','').split(',')

            
        #quartos
        quartos = anuncio.find('span',{'itemprop':'numberOfRooms'})
        print(quartos.get_text())
        if quartos == None:
            card['quarto-min'], card['quarto-max'] = '0','0' 
        elif '-' in quartos.get_text() :
            card['quarto-min'], card['quarto-max'], *outro = quartos.get_text().replace(' ','').replace('\n','').split('-')
        else:        
           card['quarto-min'], card['quarto-max'] = quartos.get_text().replace(' ','').replace('\n',''),quartos.get_text().replace(' ','').replace('\n','')    
        
        
        #vagas
        garagem = anuncio.find('li',{'class':'feature__item text-small js-parking-spaces'})
        if garagem == None:
            card['garagem-min'], card['garagem-max'] = '0' , '0'
        elif '-' in garagem.get_text() :
            card['garagem-min'], card['garagem-max'] = garagem.get_text().replace(' ','').replace('\n','').split('-')
        else:    
            card['garagem-min'], card['garagem-max'] = garagem.get_text().replace(' ','').replace('\n','') , garagem.get_text().replace(' ','').replace('\n','')

        
            
        #banheiros
        banheiro = anuncio.find('span',{'itemprop':'numberOfBathroomsTotal'})
        if banheiro == None:
            card['banheiro-min'], card['banheiro-max'] = '0','0'
        elif '-' in banheiro.get_text() :
            card['banheiro-min'], card['banheiro-max'] = banheiro.get_text().replace(' ','').replace('\n','').split('-')
        else:    
            card['banheiro-min'], card['banheiro-max'] = banheiro.get_text().replace(' ','').replace('\n',''),banheiro.get_text().replace(' ','').replace('\n','')

          
        #metragem
        metro = anuncio.find('span',{'itemprop':'floorSize'})
        if metro == None:
            card['area(m2)-min'], card['area(m2)-max'] = '0','0'
        elif '-' in metro.get_text() :
            card['area(m2)-min'], card['area(m2)-max'], *outro = metro.get_text().replace(' ','').replace('\n','').replace('m²','').split('-')
        else:
            card['area(m2)-min'], card['area(m2)-max'] = metro.get_text().replace(' ','').replace('\n','').replace('m²',''),metro.get_text().replace(' ','').replace('\n','').replace('m²','')

            
        #IPTU
        iptu = anuncio.find('li',{'class':'card-price__item iptu text-regular'})
        if iptu == None:
            card['iptu'] = int('0')
        else :
            card['iptu'] = int(iptu.get_text().replace('R$','').replace('\n','').replace('.','').replace('IPTU','').replace(' ',''))

                          
            
        #condominio
        condominio = anuncio.find('li',{'class':'card-price__item condominium text-regular'})
        if condominio == None:
            card['condominio'] = int('0')
        else :
            card['condominio'] = int(condominio.get_text().replace('condomínioR$','').replace(' ','').replace('.',''))
        
        #tipo
        tipo = anuncio.find('small')      
        if tipo == None:
            card['novidade'] = str('Nenhuma')
        else :
            card['novidade'] = str(tipo.get_text())
            
        cards.append(card)

        
dataset = pd.DataFrame(cards)
dataset.to_csv('./dados/dataset-casas-geral-rj-teste.csv', sep=';', index=False, encoding='utf-8-sig')
dataset


        4 
        

        3 - 5 
        

        5 
        

        6 
        

        5 
        

        4 
        

        3 
        

        2 
        

        3 
        

        3 
        

        2 
        

        3 
        

        4 
        

        5 
        

        3 
        

        5 
        

        4 
        

        6 
        

        3 
        

        3 
        

        4 
        

        3 
        

        3 
        

        3 
        


,valor,descricao,local,bairro/cidade,quarto-min,quarto-max,garagem-min,garagem-max,banheiro-min,banheiro-max,area(m2)-min,area(m2)-max,iptu,condominio,novidade
0,6850000,Casa única no Alto Jardim Botânico! Uma casa ...,Rua Peri,Jardim Botânico,4,4,2,2,8,8,508,610,0,0,Nenhuma
1,1999000,O Condomínio Residencial Camboata está a pouco...,Rua Geógrafo Amora,Piratininga,3,5,3,4,4,6,400,700,0,0,Nenhuma
2,2990000,Mansões Rio Mar. Excelentes projetos de casas ...,Avenida das Américas,Barra da Tijuca,5,5,3,3,6,6,460,540,0,0,Nenhuma
3,4400000,"Casa dividida em sala de estar, sala de jantar...",Urca,Rio de Janeiro,6,6,2,2,2,2,266,266,47,0,Nenhuma
4,2100000,Freguesia. Casa Linear no Condomínio Eldorado ...,Rua Professor Sílvio Fialho,Freguesia- Jacarepaguá,5,5,5,5,6,6,600,600,7500,960,Nenhuma
5,7500000,Esta linda casa com 1300m² de terreno e 900m² ...,Rua Senador Simonsen,Jardim Botânico,4,4,6,6,11,11,900,900,7800,0,Nenhuma
6,350000,A casa no bairro Campo Grande possui 130 metro...,Campo Grande,Rio de Janeiro,3,3,1,1,4,4,130,130,0,0,Nenhuma
7,239000,Linda casa para morar em Campo Grande em otima...,Rua Maurício do Vale,Cosmos,2,2,1,1,2,2,78,78,0,0,Nenhuma
8,650000,"Juntinho a Rua borja Reis com comércio, conduç...",Água Santa,Rio de Janeiro,3,3,4,4,6,6,160,160,0,0,Nenhuma
9,250000,O apartamento no bairro Campo Grande tem 130 m...,Campo Grande,Rio de Janeiro,3,3,1,1,4,4,130,130,0,0,Nenhuma
